In [1]:
# prepare the key for openai and pubmed
import os

# E2B sandbox API
with open("e2b.key", "r") as f:
    E2B_API_KEY = f.read()
    os.environ["E2B_API_KEY"] = E2B_API_KEY

# OpenAI API key
with open("openai.key", "r") as f:
    OPENAI_API_KEY_GLOBAL = f.read()
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY_GLOBAL

#  Pubmed API key
with open("pubmed.key", "r") as f:
    DEFAULT_PUBMED_API_KEY = f.read()
    os.environ["PUBMED_API_KEY"] = DEFAULT_PUBMED_API_KEY

In [2]:
# test the apis
from trialmind.api import SearchQueryGeneration
api = SearchQueryGeneration()
terms = api.run(
    population = "Patients with cancer undergoing treatment with immune checkpoint inhibitors (ICIs)",
    intervention = "The occurrence of immune-related adverse events (irAEs)",
    comparator = "Patients who did not develop irAEs",
    outcome = "Overall survival (OS) and progression-free survival (PFS)",
    llm = "openai-gpt-4o",
)
print(terms)

{'conditions': ['skin cancer', 'kidney cancer', 'renal cell carcinoma', 'bladder cancer', 'lung carcinoma', 'lung cancer', 'uterine cancer', 'melanoma', 'endometrial cancer', 'urothelial carcinoma'], 'treatments': ['immune checkpoint inhibitors', 'ICIs', 'PD-(L)1 inhibitors', 'PD-1 inhibitors', 'steroids', 'corticosteroids', 'programmed death-1 inhibitors', 'programmed death-ligand 1 inhibitors', 'immunosuppressants', 'immune suppressants'], 'outcomes': ['immune-related adverse events', 'OS', 'DCR', 'overall survival', 'disease control rate', 'ORR', 'PFS', 'progression-free survival', 'irAEs', 'objective response rate']}


In [3]:
# run the generated queries
import os
from trialmind.pubmed import PubmedAPIWrapper
search_api = PubmedAPIWrapper()
tmp_inputs = {
        "page_size": 100,
        "keyword_map": terms,
        "keywords": {
            "OPERATOR": 'AND'
        }
}
pubmed_api_key=os.getenv("PUBMED_API_KEY", None)
response = search_api.build_search_query_and_get_pmid(tmp_inputs, api_key=pubmed_api_key)

In [5]:
# retrieve the tabular format of the papers
from trialmind.pubmed import pmid2papers
df_papers = pmid2papers(pmid_list=response[0], api_key=pubmed_api_key)
df_papers[0]

,PMID,Journal,Year,Month,Day,Title,Publication Type,Authors,Abstract
0,38860089,Cureus,2024,May,,KRAS as a Prognostic and Predictive Marker in ...,"Journal Article, Review","Sheereen Fatima, Nirav Pansuriya, Alisha Lakha...",Metastatic non-small cell lung cancer (NSCLC) ...
1,38883368,American journal of translational research,2024,,,Comprehensive pan-cancer analysis reveals VSIR...,Journal Article,"Jun Pan, Ihsanullah Mahsud, Moeen Ul Haq, Salm...","Being a checkpoint, the expression level of V-..."
2,38863868,Clinical case reports,2024,Jun,,Immune checkpoint inhibitor-induced myasthenia...,"Case Reports, Journal Article","Arjun Basnet, Nava Raj Sharma, Sudarshan Gauta...",Immune checkpoint inhibitors can rarely lead t...
3,38865375,PloS one,2024,,,Efficacy and safety of concurrent immune check...,"Journal Article, Systematic Review, Meta-Analysis","Ran Cui, Yun Li, Xinlin Yu, Chun Wei, Ou Jiang",The recent usage of immunotherapy combined wit...
4,38870979,The Lancet. Respiratory medicine,2024,Jun,10,Garsorasib in patients with KRAS,Journal Article,"Ziming Li, Xiaomin Dang, Dingzhi Huang, Shi Ji...","Garsorasib (D-1553; InventisBio, Shangai, Chin..."
...,...,...,...,...,...,...,...,...,...
95,38886756,Journal of experimental & clinical cancer rese...,2024,Jun,18,Intravesical instillation-based mTOR-STAT3 dua...,Journal Article,"Dae Hoon Lee, Jung Ki Yoo, Ki Hwan Um, Wootae ...",Recent intravesical administration of adenovir...
96,38888461,Immunotherapy,2024,Jun,18,Pancreas-specific immune-related adverse event...,Journal Article,"Liu Jia, Shi Yuequan, Fang Jian, Lu Hongmin, S...",Immune-related adverse events (irAEs) are one ...
97,38911864,Frontiers in immunology,2024,,,Serum albumin and derived neutrophil-to-lympho...,Journal Article,"Zhanpeng Kuang, Jessica Miao, Xiaoli Zhang",Immune checkpoint inhibitors (ICIs) have resha...
98,38890738,Journal of translational medicine,2024,Jun,18,Mutational variant allele frequency profile as...,Journal Article,"Ruyun Gao, Ning Lou, Lin Li, Tongji Xie, Puyua...",Identifying new biomarkers for predicting immu...
